In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Se carga el set de entrenamiento

In [50]:
prop = pickle.load(open("Data.p","rb"))

In [51]:
prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72474 entries, 0 to 72473
Data columns (total 35 columns):
id                            72474 non-null object
created_on                    72474 non-null object
operation                     72474 non-null object
property_type                 72474 non-null object
place_name                    72446 non-null object
place_with_parent_names       72474 non-null object
country_name                  72474 non-null object
state_name                    72474 non-null object
geonames_id                   62030 non-null float64
lat-lon                       48174 non-null object
lat                           48174 non-null float64
lon                           48174 non-null float64
price                         72474 non-null float64
currency                      72473 non-null object
price_aprox_local_currency    72474 non-null float64
price_aprox_usd               72474 non-null float64
surface_total_in_m2           72474 non-null float64


In [52]:
propTOT = prop
#completo las superficies totales con las cubiertas
propTOT['surface_total_in_m2'] = propTOT.apply(lambda row: row[12] if(np.all(pd.isnull(row[11]))) else row[11], axis=1)
#cortamos por precios
propTOT = propTOT[(propTOT['price_usd_per_m2']<8000)]

## Division del set de Entrenamiento

In [53]:
#se formatea lat y lon
p = propTOT[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]
p.dropna(inplace = True)

p['lat'] = p.apply(lambda row: row[4][0], axis=1)

p['lon'] = p.apply(lambda row: row[4][1], axis=1)

p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Se le asigna un valor numerico al tipo de propiedad

In [54]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [55]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [56]:
def asignarPV(name, dic):
    return dic[name]   

In [57]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [58]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*100
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [59]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [60]:
pEnt = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

In [61]:
pEnt.dropna(inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Se carga el set de Prueba

# PRUEBA Random Forest Regresivo

In [62]:
proper = pickle.load(open("porpiedadesPruebaConColTransGas.p", "rb"))

#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

#completo las superficies totales con las cubiertas
p['surface_total_in_m2'] = p.apply(lambda row: row[3] if(np.all(pd.isnull(row[2]))) else row[2], axis=1)

pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [63]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pEnt[['property_type','surface_total_in_m2',
        'place_value']],pEnt[['price_aprox_usd']],test_size=0.0)

In [64]:
#definir el modelo a probar
model = RandomForestRegressor(n_estimators=5000, max_features='log2', min_samples_leaf=2, min_samples_split=4, bootstrap=True, oob_score=True)
n = 5000

#fit model on training dataset
model.fit(Xtrn, Ytrn['price_aprox_usd'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [65]:
pPru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 9 columns):
property_type                14166 non-null int64
surface_total_in_m2          13392 non-null float64
lat                          12586 non-null float64
lon                          12586 non-null float64
Cant_ColeYUniv               12586 non-null float64
Cant_LocalesGastronomicos    12586 non-null float64
Cant_ParadasTransporte       12586 non-null float64
place_value                  14157 non-null float64
price_aprox_usd              8686 non-null float64
dtypes: float64(8), int64(1)
memory usage: 996.1 KB


In [66]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imp = imp.fit(Xtrn)

pPru_imp = imp.transform(pPru[['property_type','surface_total_in_m2',
        'place_value']])

In [67]:
pPru['price_usd'] = model.predict(pPru_imp)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
result = pd.DataFrame()

In [69]:
result["id"] = proper["id"]

In [70]:
result["price_usd"] = pPru['price_usd']

In [71]:
result.head()

id      price_usd
0  2316505   64546.427041
1  2320880  210014.695473
2  2321742  216459.265188
3  2276796   73975.570421
4  2330364   65491.657711

In [72]:
result.to_csv(index=False, path_or_buf="results8PhiAlpha.csv")